This Jupyter Notebook demonstrates how to use YOLOv3 (You Only Look Once) for real-time object detection using a laptop's webcam. The code captures video from the webcam, processes each frame using the YOLOv3 model, and displays the detected objects in real-time.

First, we need to import the necessary libraries. We'll use OpenCV for video capture and image processing, and NumPy for handling arrays.

In [ ]:
import cv2
import numpy as np

We need to load the YOLOv3 model using the weights and configuration files. We also need to load the COCO class labels.

In [ ]:
# Load YOLOv3 weights and configuration
weights_path = 'yolov3.weights'
config_path = 'yolov3.cfg'
coco_names_path = 'coco.names'

# Load YOLO model
net = cv2.dnn.readNet(weights_path, config_path)

# Load COCO class labels
with open(coco_names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

We'll capture video from the default webcam.

In [ ]:
cap = cv2.VideoCapture(0)  # 0 for the default webcam

Define the parameters for image processing.

In [ ]:
scale_factor = 1/255.0  # Normalizes pixel values
size = (416, 416)  # Resize the image to 416x416 as required by YOLO
swap_rb = True  # Swap red and blue channels
crop = False

Retrieve the output layer names for YOLO.

In [ ]:
layer_names = net.getLayerNames()  # Retrieves names of all layers in the model
output_layers_indices = net.getUnconnectedOutLayers() # Gets indices of output layers
output_layers = [layer_names[i - 1] for i in output_layers_indices]

The main real-time detection loop starts by processing each frame from the webcam and perform object detection. First, the code captures a frame from the webcam. The `ret` value is a boolean indicating whether the frame was successfully read. If `ret` is `False`, the loop breaks, meaning the video capture has ended or there was an issue with reading the frame.

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

Gets the dimensions of the captured frame. `height` and `width` are used to scale bounding boxes and text correctly on the frame.

In [ ]:
    height, width, _ = frame.shape

It then converts the image into a blob suitable for YOLO input. This involves resizing the image to 416x416 (as required by YOLO), normalizing pixel values, and swapping color channels if necessary. And then sets the prepared blob as input to the YOLO network.

In [ ]:
    blob = cv2.dnn.blobFromImage(frame, scalefactor=scale_factor, size=size, swapRB=swap_rb, crop=crop)
    net.setInput(blob) 


Next, performs a forward pass through the network and retrieves the outputs from the specified output layers. `outs` contains the raw detection data from YOLO.

In [ ]:
    outs = net.forward(output_layers) 

    class_ids = [] # Lists to store the IDs of detected classes.
    confidences = [] # Their confidence scores.
    boxes = [] # Their bounding box coordinates.

Prints the shape of each output layer. YOLOv3 generates multiple outputs, and their shapes can vary. 


In [ ]:
    for out in outs:
        print(f"Output layer shape: {out.shape}")

Checks if the output is a 2D array, as for YOLOv3, each detection is expected to have 85 values: 4 for bounding box coordinates (x_center, y_center, width, height), 1 for objectness score, and 80 for class scores.

In [ ]:
        if len(out.shape) == 2:
            num_detections, num_values = out.shape

Extracts bounding box coordinates, objectness score, and class scores.

In [ ]:
            if num_values == 85:
                for det in out:
                    x_center = det[0]
                    y_center = det[1]
                    w = det[2]
                    h = det[3]
                    objectness = det[4]
                    scores = det[5:]

Converts normalized bounding box coordinates to pixel values based on the frame dimensions.

In [ ]:
                    center_x = int(x_center * width)
                    center_y = int(y_center * height)
                    w = int(w * width)
                    h = int(h * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

Filters detections based on objectness and confidence scores. If the objectness and confidence exceed thresholds, the detection is considered valid and added to the lists.

In [ ]:
                    if objectness > 0.5:
                        class_id = np.argmax(scores)
                        confidence = scores[class_id]
                        if confidence > 0.5:  # Confidence threshold
                            boxes.append([x, y, w, h])
                            confidences.append(float(confidence))
                            class_ids.append(class_id)
            else:
                print(f"Unexpected number of values per detection: {num_values}")

For 3D outputs (i.e., multiple detections per layer), processes each detection similarly as the 2D outputs. Also handles cases where the output shape does not match the expected formats, printing an error message in both outputs cases.

In [ ]:
        elif len(out.shape) == 3: 
            for detection in out:
                if detection.shape[1] == 85:
                    for det in detection:
                        # Extract values from the detection array
                        x_center = det[0]
                        y_center = det[1]
                        w = det[2]
                        h = det[3]
                        objectness = det[4]
                        scores = det[5:]

                        # Convert to image coordinates
                        center_x = int(x_center * width)
                        center_y = int(y_center * height)
                        w = int(w * width)
                        h = int(h * height)
                        x = int(center_x - w / 2)
                        y = int(center_y - h / 2)

                        # Filter based on objectness and confidence
                        if objectness > 0.5:
                            class_id = np.argmax(scores)
                            confidence = scores[class_id]
                            if confidence > 0.5:  # Confidence threshold
                                boxes.append([x, y, w, h])
                                confidences.append(float(confidence))
                                class_ids.append(class_id)
                else:
                    print(f"Unexpected number of values per detection: {detection.shape[1]}")
        else:
            print(f"Unexpected output shape: {out.shape}")

Applies non-max suppression to eliminate redundant overlapping bounding boxes. Only the box with the highest confidence is retained.

In [ ]:
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

For each valid detection, draws a bounding box and label on the frame. The label includes the class name and confidence score.

In [ ]:
    if len(indices) > 0:
        indices = indices.flatten()
        for i in indices:
            x, y, w, h = boxes[i]
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

Shows the current frame with detected objects in a window.

In [ ]:
    cv2.imshow('YOLO Real-Time Detection', frame)

Allows exiting the loop by pressing the 'q' key.

In [ ]:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

After the loop ends, release the webcam and close all the display window.

In [ ]:
cap.release()
cv2.destroyAllWindows()